In [1]:
!pip install mediapipe

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 18.5 MB/s eta 0:00:00


In [2]:
import cv2
import os
import mediapipe as mp

In [3]:
input_folder = "/content/drive/MyDrive/CareVision/Dataset/Images/train"  # Replace with your folder path
output_folder = "/content/drive/MyDrive/CareVision/Dataset/Images/train(RGB+SK)"  # Replace with your folder path
metadata_file = "/content/drive/MyDrive/CareVision/Dataset/Images/pose_metadata.csv"

In [4]:
# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

os.makedirs(output_folder, exist_ok=True)

# Load Pose Estimator
with mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5) as pose:
    for filename in os.listdir(input_folder):
        if filename.endswith((".jpg", ".png", ".jpeg")):
            image_path = os.path.join(input_folder, filename)
            image = cv2.imread(image_path)

            if image is None:
                print(f"Skipping {filename}, unable to load.")
                continue

            # Convert Image to RGB (for MediaPipe)
            rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # Process Image for Pose Detection
            results = pose.process(rgb_image)

            # Draw Pose Skeleton on Image
            if results.pose_landmarks:
                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            # Save the Processed Image
            output_path = os.path.join(output_folder, filename)
            cv2.imwrite(output_path, image)
            print(f"Saved: {output_path}")

print("Processing complete!")

Saved: /content/drive/MyDrive/CareVision/Dataset/Images/train(RGB+SK)/fall097.jpg
Saved: /content/drive/MyDrive/CareVision/Dataset/Images/train(RGB+SK)/fall017.jpg
Saved: /content/drive/MyDrive/CareVision/Dataset/Images/train(RGB+SK)/fall137.jpg
Saved: /content/drive/MyDrive/CareVision/Dataset/Images/train(RGB+SK)/fall036.jpg
Saved: /content/drive/MyDrive/CareVision/Dataset/Images/train(RGB+SK)/fall104.jpg
Saved: /content/drive/MyDrive/CareVision/Dataset/Images/train(RGB+SK)/fall136.jpg
Saved: /content/drive/MyDrive/CareVision/Dataset/Images/train(RGB+SK)/fall152.jpg
Saved: /content/drive/MyDrive/CareVision/Dataset/Images/train(RGB+SK)/fall106.jpg
Saved: /content/drive/MyDrive/CareVision/Dataset/Images/train(RGB+SK)/fall044.jpg
Saved: /content/drive/MyDrive/CareVision/Dataset/Images/train(RGB+SK)/fall155.jpg
Saved: /content/drive/MyDrive/CareVision/Dataset/Images/train(RGB+SK)/fall138.jpg
Saved: /content/drive/MyDrive/CareVision/Dataset/Images/train(RGB+SK)/fall030.jpg
Saved: /content/

KeyboardInterrupt: 

In [ ]:
import cv2
import os
import mediapipe as mp
import numpy as np
import pandas as pd

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils


os.makedirs(output_folder, exist_ok=True)

# Function to calculate angle between three points
def calculate_angle(a, b, c):
    a = np.array(a)  # Point A
    b = np.array(b)  # Point B (Joint)
    c = np.array(c)  # Point C

    ba = a - b  # Vector BA
    bc = c - b  # Vector BC

    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))  # Avoid floating-point errors
    return np.degrees(angle)  # Convert to degrees

# Initialize Data Storage
pose_data = []

# Load Pose Estimator
with mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5) as pose:
    for filename in os.listdir(input_folder):
        if filename.endswith((".jpg", ".png", ".jpeg")):
            image_path = os.path.join(input_folder, filename)
            image = cv2.imread(image_path)

            if image is None:
                print(f"Skipping {filename}, unable to load.")
                continue

            # Convert Image to RGB (for MediaPipe)
            rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # Process Image for Pose Detection
            results = pose.process(rgb_image)

            # Draw Pose Skeleton on Image
            if results.pose_landmarks:
                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

                # Extract Landmarks
                landmarks = results.pose_landmarks.landmark
                keypoints = {i: (lm.x, lm.y, lm.z) for i, lm in enumerate(landmarks)}

                # Define Joint Angles
                try:
                    elbow_angle = calculate_angle(keypoints[mp_pose.PoseLandmark.LEFT_SHOULDER],
                                                  keypoints[mp_pose.PoseLandmark.LEFT_ELBOW],
                                                  keypoints[mp_pose.PoseLandmark.LEFT_WRIST])

                    knee_angle = calculate_angle(keypoints[mp_pose.PoseLandmark.LEFT_HIP],
                                                 keypoints[mp_pose.PoseLandmark.LEFT_KNEE],
                                                 keypoints[mp_pose.PoseLandmark.LEFT_ANKLE])

                    hip_angle = calculate_angle(keypoints[mp_pose.PoseLandmark.LEFT_SHOULDER],
                                                keypoints[mp_pose.PoseLandmark.LEFT_HIP],
                                                keypoints[mp_pose.PoseLandmark.LEFT_KNEE])

                    torso_bend_angle = calculate_angle(keypoints[mp_pose.PoseLandmark.LEFT_SHOULDER],
                                                       keypoints[mp_pose.PoseLandmark.LEFT_HIP],
                                                       keypoints[mp_pose.PoseLandmark.LEFT_ANKLE])

                    # Append data to list
                    pose_data.append({
                        "filename": filename,
                        "elbow_angle": elbow_angle,
                        "knee_angle": knee_angle,
                        "hip_angle": hip_angle,
                        "torso_bend_angle": torso_bend_angle,
                        "left_shoulder_x": keypoints[mp_pose.PoseLandmark.LEFT_SHOULDER][0],
                        "left_shoulder_y": keypoints[mp_pose.PoseLandmark.LEFT_SHOULDER][1],
                        "left_hip_x": keypoints[mp_pose.PoseLandmark.LEFT_HIP][0],
                        "left_hip_y": keypoints[mp_pose.PoseLandmark.LEFT_HIP][1],
                    })

                except KeyError:
                    print(f"Skipping {filename}, pose landmarks missing.")

            # Save Processed Image
            output_path = os.path.join(output_folder, filename)
            cv2.imwrite(output_path, image)
            print(f"Saved: {output_path}")

# Convert Data to CSV
df = pd.DataFrame(pose_data)
df.to_csv(metadata_file, index=False)
print(f"Pose metadata saved to {metadata_file} ✅")

In [4]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Flatten, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50

# Image Input (RGB and Skeleton images will be passed separately)
image_input = Input(shape=(224, 224, 3), name="image_input")

# Use ResNet50 for feature extraction
base_model = ResNet50(weights="imagenet", include_top=False, input_tensor=image_input)
x = Flatten()(base_model.output)
image_features = Dense(128, activation="relu")(x)

# Joint Angles Input (Numerical Features)
pose_input = Input(shape=(8,), name="pose_input")  # Adjust size based on your CSV data
pose_features = Dense(32, activation="relu")(pose_input)
pose_features = Dense(16, activation="relu")(pose_features)

# Merge Image Features and Pose Features
merged = Concatenate()([image_features, pose_features])
merged = Dense(64, activation="relu")(merged)
output = Dense(2, activation="softmax", name="output")(merged)  # 3 classes: Fall, No Fall, Sitting

# Define Model
model = Model(inputs=[image_input, pose_input], outputs=output)

# Compile Model
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Summary of the model
model.summary()

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ image_input (InputLayer)  │ (None, 224, 224, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_pad (ZeroPadding2D) │ (None, 230, 230, 3)    │              0 │ image_input[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_conv (Conv2D)       │ (None, 112, 112, 64)   │          9,472 │ conv1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_bn                  │ (None, 112, 112, 64)   │            256 │ conv1_conv[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_relu (Activation)   │ (None, 112, 112, 64)   │              0 │ conv1_bn[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pad (ZeroPadding2D) │ (None, 114, 114, 64)   │              0 │ conv1_relu[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pool (MaxPooling2D) │ (None, 56, 56, 64)     │              0 │ pool1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_conv       │ (None, 56, 56, 64)     │          4,160 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_1_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_1_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_conv       │ (None, 56, 56, 64)     │         36,928 │ conv2_block1_1_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_2_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_2_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_conv       │ (None, 56, 56, 256)    │         16,640 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_3_conv       │ (None, 56, 56, 256)    │         16,640 │ conv2_block1_2_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├──────────────────────

 Total params: 36,443,122 (139.02 MB)

 Trainable params: 36,390,002 (138.82 MB)

 Non-trainable params: 53,120 (207.50 KB)

In [5]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Define paths for images and CSV
# image_folder = 'path_to_image_folder'
# csv_file = 'path_to_joint_angles.csv'

# Load the CSV containing joint angles and pose data
pose_data_df = pd.read_csv(metadata_file)

# Initialize lists for images, pose data, and labels
images = []
pose_data = []
labels = []

# Loop through each row in the CSV and process the corresponding image
for index, row in pose_data_df.iterrows():
    # Get the image filename (assuming the filename in CSV matches the image filename in the folder)
    img_name = row['filename']
    image_file = os.path.join(output_folder, img_name)

    # Load and preprocess the image
    img = load_img(image_file, target_size=(224, 224))  # Resize to (224, 224)
    img_array = img_to_array(img)  # Convert image to array
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = img_array / 255.0  # Normalize the image

    # Extract the joint angles and pose data from the CSV row
    angles = np.array([row['elbow_angle'], row['knee_angle'], row['hip_angle'], row['torso_bend_angle']])
    pose_coordinates = np.array([row['left_shoulder_x'], row['left_shoulder_y'], row['left_hip_x'], row['left_hip_y']])

    # Combine the image and pose data for the model
    images.append(img_array)
    pose_data.append(np.concatenate([angles, pose_coordinates]))  # Concatenate joint angles and coordinates

    # The filename is the label, so we append it as the label
    if 'not fallen' in img_name:
        labels.append(0)
    else:
        labels.append(1)

# Convert the lists to NumPy arrays
images = np.vstack(images)  # Stack all images into a single NumPy array
pose_data = np.array(pose_data)  # Convert pose data to NumPy array
labels = np.array(labels)  # Convert labels to NumPy array
labels = tf.keras.utils.to_categorical(labels, num_classes=2)  # Convert labels to one-hot encoding

# Verify the shapes
print(f"Images shape: {images.shape}")
print(f"Pose Data shape: {pose_data.shape}")
print(f"Labels shape: {labels.shape}")

Images shape: (355, 224, 224, 3)
Pose Data shape: (355, 8)
Labels shape: (355, 2)


In [6]:
model.fit([images, pose_data], labels, epochs=10)

# Or, for predictions
# predictions = model.predict([images, pose_data])

Epoch 1/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 91s 2s/step - accuracy: 0.5409 - loss: 16.6673
Epoch 2/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 31s 259ms/step - accuracy: 0.7407 - loss: 2.5516
Epoch 3/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 3s 260ms/step - accuracy: 0.8150 - loss: 0.6088
Epoch 4/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 3s 262ms/step - accuracy: 0.9347 - loss: 0.1551
Epoch 5/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 5s 264ms/step - accuracy: 0.9716 - loss: 0.0866
Epoch 6/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 3s 262ms/step - accuracy: 0.9849 - loss: 0.0394
Epoch 7/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 3s 264ms/step - accuracy: 0.9624 - loss: 0.0957
Epoch 8/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 5s 268ms/step - accuracy: 0.9768 - loss: 0.0651
Epoch 9/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 5s 265ms/step - accuracy: 0.9326 - loss: 0.2335
Epoch 10/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 5s 270ms/step - accuracy: 0.9622 - loss: 0.1708


In [8]:
model.save("/content/drive/MyDrive/CareVision/Model.keras", save_format='tf')

In [12]:
model.predict([images[210:230], pose_data[210:230]])

1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step


array([[1.0000e+00, 0.0000e+00],
       [1.0000e+00, 0.0000e+00],
       [1.0000e+00, 1.4013e-45],
       [1.0000e+00, 4.2039e-45],
       [1.0000e+00, 0.0000e+00],
       [1.0000e+00, 0.0000e+00],
       [1.0000e+00, 0.0000e+00],
       [1.0000e+00, 0.0000e+00],
       [1.0000e+00, 0.0000e+00],
       [1.0000e+00, 2.5223e-44],
       [1.0000e+00, 1.2355e-41],
       [1.0000e+00, 1.8217e-44],
       [1.0000e+00, 0.0000e+00],
       [1.0000e+00, 2.9427e-43],
       [1.0000e+00, 0.0000e+00],
       [1.0000e+00, 0.0000e+00],
       [1.0000e+00, 2.8026e-45],
       [1.0000e+00, 0.0000e+00],
       [1.0000e+00, 7.2868e-44],
       [1.0000e+00, 6.7276e-42]], dtype=float32)

In [13]:
labels

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.

In [ ]:
tf.keras.utils.to_categorical(labels, num_classes=2)

In [ ]:
img_name = "not fallen002.jpg"
if 'not fallen' in img_name:
    print('not_fallen')
else:
    print('fall')